In [37]:
using Statistics
using Flux, Flux.Optimise
using Flux: onehotbatch, onecold
using Flux: crossentropy, Momentum
using Base.Iterators: partition
using Metal
using MLDatasets
using MLUtils
using Images.ImageCore
using BenchmarkTools: @btime

In [55]:
Flux.OneHotMatrix(X.targets, 10) |> gpu

10×50000 OneHotMatrix(::MtlVector{Int64, Private}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  1  1
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅     ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [62]:
# Prepare data
X = CIFAR10(:train)
labels = Flux.OneHotMatrix(X.targets, 10) |> gpu
imgs = X.features |> gpu

# Load data to gpu and cpu
train_cpu = [(imgs[:,:,:,i], labels[:,i]) for i in partition(1:50000, 1000)] |> cpu
train_gpu = [(imgs[:,:,:,i], labels[:,i]) for i in partition(1:50000, 1000)] |> gpu
;

In [63]:
# Define neural networks for both gpu and cpu
m_gpu = Chain(
  Conv((5,5), 3=>16, relu),
  MaxPool((2,2)),
  Conv((5,5), 16=>8, relu),
  MaxPool((2,2)),
  flatten,
  Dense(200, 120),
  Dense(120, 84),
  Dense(84, 10),
  softmax) |> gpu

Chain(
  Conv((5, 5), 3 => 16, relu),          # 1_216 parameters
  MaxPool((2, 2)),
  Conv((5, 5), 16 => 8, relu),          # 3_208 parameters
  MaxPool((2, 2)),
  MLUtils.flatten,
  Dense(200 => 120),                    # 24_120 parameters
  Dense(120 => 84),                     # 10_164 parameters
  Dense(84 => 10),                      # 850 parameters
  NNlib.softmax,
)                   # Total: 10 arrays, 39_558 parameters, 1.758 KiB.

In [64]:
m_cpu = Chain(
  Conv((5,5), 3=>16, relu),
  MaxPool((2,2)),
  Conv((5,5), 16=>8, relu),
  MaxPool((2,2)),
  flatten,
  Dense(200, 120),
  Dense(120, 84),
  Dense(84, 10),
  softmax) |> cpu

Chain(
  Conv((5, 5), 3 => 16, relu),          # 1_216 parameters
  MaxPool((2, 2)),
  Conv((5, 5), 16 => 8, relu),          # 3_208 parameters
  MaxPool((2, 2)),
  MLUtils.flatten,
  Dense(200 => 120),                    # 24_120 parameters
  Dense(120 => 84),                     # 10_164 parameters
  Dense(84 => 10),                      # 850 parameters
  NNlib.softmax,
)                   # Total: 10 arrays, 39_558 parameters, 155.852 KiB.

In [65]:
# Define loss and optimizer
loss_gpu(m, x, y) = sum(crossentropy(m(x), y))
opt_gpu = Momentum(0.01)

loss_cpu(x, y) = sum(crossentropy(m_cpu(x), y))
opt_cpu = Momentum(0.01)

Momentum(0.01, 0.9, IdDict{Any, Any}())

In [66]:
# Set number of training iterations
epochs = 1

1

In [68]:
for epoch in 1:epochs
    for (x, y) in train_gpu
        grads = gradient(m -> loss_gpu(m, x, y), m_gpu)
        Flux.update!(opt_state, model, grads[1])
    end
end

CompositeException: TaskFailedException

    nested task error: TaskFailedException
    
        nested task error: Scalar indexing is disallowed.
        Invocation of getindex resulted in scalar indexing of a GPU array.
        This is typically caused by calling an iterating implementation of a method.
        Such implementations *do not* execute on the GPU, but very slowly on the CPU,
        and therefore should be avoided.
        
        If you want to allow scalar iteration, use `allowscalar` or `@allowscalar`
        to enable scalar iteration globally or for the operations in question.
        Stacktrace:
          [1] error(s::String)
            @ Base ./error.jl:35
          [2] errorscalar(op::String)
            @ GPUArraysCore ~/.julia/packages/GPUArraysCore/GMsgk/src/GPUArraysCore.jl:155
          [3] _assertscalar(op::String, behavior::GPUArraysCore.ScalarIndexing)
            @ GPUArraysCore ~/.julia/packages/GPUArraysCore/GMsgk/src/GPUArraysCore.jl:128
          [

In [46]:
# GPU benchmark
# @btime for epoch = 1:epochs
  for d in train_gpu
    gs = gradient(Flux.params(m_gpu)) do
      l = loss_gpu(d...)
    end
    update!(opt_gpu, params(m_gpu), gs)
  end
# end

CompositeException: TaskFailedException

    nested task error: TaskFailedException
    
        nested task error: Scalar indexing is disallowed.
        Invocation of getindex resulted in scalar indexing of a GPU array.
        This is typically caused by calling an iterating implementation of a method.
        Such implementations *do not* execute on the GPU, but very slowly on the CPU,
        and therefore should be avoided.
        
        If you want to allow scalar iteration, use `allowscalar` or `@allowscalar`
        to enable scalar iteration globally or for the operations in question.
        Stacktrace:
          [1] error(s::String)
            @ Base ./error.jl:35
          [2] errorscalar(op::String)
            @ GPUArraysCore ~/.julia/packages/GPUArraysCore/GMsgk/src/GPUArraysCore.jl:155
          [3] _assertscalar(op::String, behavior::GPUArraysCore.ScalarIndexing)
            @ GPUArraysCore ~/.julia/packages/GPUArraysCore/GMsgk/src/GPUArraysCore.jl:128
          [

In [44]:
# CPU benchmark
for epoch = 1:epochs
    for d in train_cpu
      gs = gradient(Flux.params(m_cpu)) do
        l = loss_cpu(d...)
      end
      update!(opt_cpu, Flux.params(m_cpu), gs)
    end
  end